<a href="https://colab.research.google.com/github/chom5621/ESAA/blob/main/OB_1220_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C_%EC%B0%A8%EC%9B%90%EC%B6%95%EC%86%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 데이터셋의 차원을 축소하는 목적은 무엇인가요? 대표적인 단점은 무엇인가요?

**목적**

**- 훈련 데이터의 훈련 속도를 높일 수 있다.**

**- 데이터 시각화에 유용하며 이를 통해 중요한 통찰을 얻을 수 있다.**




**단점**

**- 차원을 축소시키면 일부 정보가 유실된다. 그래서 훈련 속도가 빨라질 수는 있지만 시스템의 성능이 조금 나빠질 수 있다.**

**- 작업 파이프라인이 조금 더 복잡하게 되고 유지 관리가 어려워진다.** 

2. 기본 PCA, 점진적 PCA, 랜덤 PCA,커널 PCA는 어느 경우에 사용될까요?

**기본 PCA**

**- 압축을 위해 사용한다. 차원을 축소하여, 훈련 세트의 크기를 줄인다.**

**점진적 PCA**

**- SVD 알고리즘을 실행하기 위해 전체 훈련 메모리에 올려야 할 때 사용한다.**

**- 훈련 세트의 크기가 클 때 유용하여, 새로운 데이터를 실시간으로 적용할 수 있다.**

**랜덤 PCA**

**- 알고리즘의 계산 복잡도를 줄이기 위해 사용한다.**

**커널 PCA**

**- 복잡한 비선형 투영을 수행할 수 있다.**

3. MNIST 데이터셋을 로드하고 훈련세트와 테스트세트로 분할하겠습니다.

# 모듈 설정 및 데이터 불러오기

In [1]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# MNIST 불러오기
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)

# 훈련세트와 테스트세트로 분할

In [2]:
# 첫 60000개는 훈련을 위한 샘플, 나머지 10000개는 테스트용으로 분할
X_train = mnist['data'][:60000]
y_train = mnist['target'][:60000]

X_test = mnist['data'][60000:]
y_test = mnist['target'][60000:]

3.1 이 데이터셋을 랜덤 포레스트 분류기로 훈련시키고 시간이 얼마나 오래걸리는지 시간을 잰 후, 테스트 세트로 만들어진 모델의 정확도를 평가합니다.

In [3]:
# 랜덤포레스트로 훈련시키는 코드
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [4]:
# 훈련 시간을 재는 코드입니다.
import time

t0 = time.time()
rnd_clf.fit(X_train, y_train)
t1 = time.time()

print("훈련 시간: {:.2f}s".format(t1 - t0))

훈련 시간: 49.19s


In [5]:
# 정확도 평가 코드입니다.
from sklearn.metrics import accuracy_score

y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9705

**3.2 문제 : 데이터셋을 PCA를 사용해 설명된 분산이 95%가 되도록 차원을 축소하세요**

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

**3.3 문제 : 이 축소된 데이터셋에 위와 같이 새로운 랜덤포레스트 분류기를 훈련시키고 얼마나 오래걸리는지 확인합니다. 훈련 시간이 빨라졌나요?**

In [7]:
t0 = time.time()
rnd_clf.fit(X_reduced, y_train)
t1 = time.time()

print("훈련 시간: {:.2f}s".format(t1 - t0))

훈련 시간: 121.93s


**3.4 문제: 이 테스트 세트에서 이 분류기의 정확도를 평가해보세요. 이전 분류기와 비교해서 어떤가요?**

In [8]:
X_test_reduced = pca.transform(X_test)

y_pred_reduced = rnd_clf.predict(X_test_reduced)

accuracy_score(y_test, y_pred_reduced)

0.9481

3.5 소프트 맥스 회귀를 사용해서도 같은 방식으로 실험을 해봅니다.

In [9]:
# 차원축소 이전의 코드
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression(multi_class="multinomial", solver="lbfgs", random_state=42)
t0 = time.time()
log_clf.fit(X_train, y_train)
t1 = time.time()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [10]:
print("훈련 시간: {:.2f}s".format(t1 - t0))

훈련 시간: 43.79s


In [11]:
y_pred = log_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9255

**3.6 문제 : 3.2에서 축소된 데이터 셋을 사용해 소프트맥스 회귀모델을 훈련시켜보세요. 그리고 훈련시간을 확인하세요**

In [12]:
t0 = time.time()
log_clf.fit(X_reduced, y_train)
t1 = time.time()

print("훈련 시간: {:.2f}s".format(t1 - t0))

훈련 시간: 13.84s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


**3.7 문제 : 모델의 정확도도 확인해보세요**

In [13]:
y_pred_reduced = log_clf.predict(X_test_reduced)

accuracy_score(y_test, y_pred_reduced)

0.9201

**3.8 문제 : 랜덤포레스트 모형의 훈련시간과 정확도, 소프트맥스 모형의 훈련시간과 정확도를 비교해보세요.**

**훈련세트 차원축소 결과, 랜덤포레스트 모형의 훈련시간은 오히려 2배 이상 증가했고, 정확도 또한 떨어지는 것을 확인할 수 있었다. 차원 축소는 언제나 훈련 시간을 줄여주지 못한다는 것을 알 수 있었다.**

**반면 소프트맥스 모형에서는 차원축소된 데이터셋의 훈련시간이 매우 감소한 것을 확인할 수 있었으며, 그 대신 차원축소로 인해 정확도가 떨어지는 단점도 찾을 수 있었다.**